# Chapter 7. BLAST

Dealing with BLAST can be split up into two steps, both of which can be done from within Biopython.
Firstly, running BLAST for your query sequence(s), and getting some output. Secondly, parsing the BLAST
output in Python for further analysis.

Your first introduction to running BLAST was probably via the NCBI web-service. In fact, there are
lots of ways you can run BLAST, which can be categorized in several ways. The most important distinction
is running BLAST locally (on your own machine), and running BLAST remotely (on another machine,
typically the NCBI servers). We’re going to start this chapter by invoking the NCBI online BLAST service
from within a Python script.

NOTE: The following Chapter 8 describes `Bio.SearchIO`, an experimental module in Biopython. We
intend this to ultimately replace the older Bio.Blast module, as it provides a more general framework
handling other related sequence searching tools as well. However, until that is declared stable, for production
code please continue to use the `Bio.Blast` module for dealing with NCBI BLAST.

## 7.1 Running BLAST over the Internet
We use the function qblast() in the `Bio.Blast.NCBIWWW` module to call the online version of BLAST. This has three non-optional arguments:
* The first argument is the blast program to use for the search, as a lower case string. The options and descriptions of the programs are available at https://blast.ncbi.nlm.nih.gov/Blast.cgi. Currently qblast only works with blastn, blastp, blastx, tblast and tblastx.
* The second argument specifies the databases to search against. Again, the options for this are available on the NCBI Guide to BLAST ftp://ftp.ncbi.nlm.nih.gov/pub/factsheets/HowTo_BLASTGuide.pdf.
* The third argument is a string containing your query sequence. This can either be the sequence itself, the sequence in fasta format, or an identifier like a GI number.

NCBI guidelines, from https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Web&PAGE_TYPE=BlastDocs&DOC_TYPE=DeveloperInfo state:
1. Do not contact the server more often than once every 10 seconds.
2. Do not poll for any single RID more often than once a minute.
3. Use the URL parameter email and tool, so that the NCBI can contact you if there is a problem.
4. Run scripts weekends or between 9 pm and 5 am Eastern time on weekdays if more than 50 searches will be submitted.

To fulfill the third point, one can set the `NCBIWWW.email` variable.

In [33]:
from Bio.Blast import NCBIWWW
#NCBIWWW.email = "A.N.Other@example.com"

For more about the optional BLAST arguments, we refer you to the NCBI’s own documentation, or that built into Biopython:
```
from Bio.Blast import NCBIWWW
help(NCBIWWW.qblast)
```

Note that the default settings on the NCBI BLAST website are not quite the same as the defaults on QBLAST. If you get different results, you’ll need to check the parameters (e.g., the expectation value threshold and the gap values).

For example, if you have a nucleotide sequence you want to search against the nucleotide database (nt) using BLASTN, and you know the GI number of your query sequence, you can use:

In [34]:
from Bio.Blast import NCBIWWW
result_handle = NCBIWWW.qblast("blastn", "nt", "8332116")

Alternatively, if we have our query sequence already in a FASTA formatted file, we just need to open the file and read in this record as a string, and use that as the query argument:
* To download test file : [`m_cold.fasta`](https://github.com/biopython/biopython/blob/master/Doc/examples/m_cold.fasta)

In [36]:
from Bio.Blast import NCBIWWW
fasta_string = open("m_cold.fasta").read()
result_handle = NCBIWWW.qblast("blastn", "nt", fasta_string)

We could also have read in the FASTA file as a SeqRecord and then supplied just the sequence itself:

In [37]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO
record = SeqIO.read("m_cold.fasta", format="fasta")
result_handle = NCBIWWW.qblast("blastn", "nt", record.seq)

Supplying just the sequence means that BLAST will assign an identifier for your sequence automatically.
You might prefer to use the SeqRecord object’s format method to make a FASTA string (which will include the existing identifier):

In [39]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO
record = SeqIO.read("m_cold.fasta", format="fasta")
result_handle = NCBIWWW.qblast("blastn", "nt", record.format("fasta"))

Whatever arguments you give the `qblast()` function, you should get back your results in a handle object (by default in XML format).

We need to be a bit careful since we can use `result_handle.read()` to read the BLAST output only once – calling `result_handle.read()` again returns an empty string.

In [40]:
with open("my_blast.xml", "w") as out_handle:
    out_handle.write(result_handle.read())
result_handle.close()

After doing this, the results are in the file `my_blast.xml` and the original handle has had all its data extracted (so we closed it). However, the `parse` function of the BLAST parser (described in 7.3) takes a file-handle-like object, so we can just open the saved file for input:

In [41]:
result_handle = open("my_blast.xml")

## 7.2 Running BLAST locally
### 7.2.1 Introduction
Running BLAST locally (as opposed to over the internet, see Section 7.1) has at least major two advantages:
* Local BLAST may be faster than BLAST over the internet;
* Local BLAST allows you to make your own database to search for sequences against.

Unfortunately, there are some major drawbacks too – installing all the bits and getting it setup right takes some effort:
* Local BLAST requires command line tools to be installed.
* Local BLAST requires (large) BLAST databases to be setup (and potentially kept up to date).

### 7.2.2 Standalone NCBI BLAST+
The “new” [NCBI BLAST+](https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Web&PAGE_TYPE=BlastDocs&DOC_TYPE=Download) suite was released in 2009. This replaces the old NCBI “legacy” BLAST package (see below).
This section will show briefly how to use these tools from within Python. If you have already read or tried the alignment tool examples in Section 6.5 this should all seem quite straightforward.

For example, taking a FASTA file of gene nucleotide sequences, you might want to run a BLASTX (translation) search against the non-redundant (NR) protein database. Assuming you (or your systems administrator) has downloaded and installed the [NR database](https://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/), 

* ex) Make blast db for 'swissprot'
```
$ makeblastdb -in swissprot -dbtype 'nucl' -out swissprot
$ makeblastdb -in swissprot -dbtype 'prot' -out swissprot
```

you might run:

`$ blastx -query opuntia.fasta -db nr -out opuntia.xml -evalue 0.001 -outfmt 5`

This should run BLASTX against the NR database, using an expectation cut-off value of 0.001 and produce XML output to the specified file (which we can then parse). On my computer this takes about six minutes - a good reason to save the output to a file so you can repeat any analysis as needed.

From within python we can use the subprocess module to build the command line string, and run it:

In [12]:
import subprocess
cmd = "blastp -query opuntia.fasta -db swissprot -out opuntia.xml" # "blastx -query opuntia.fasta -db nr -out opuntia.xml" in original code
cmd += " -evalue 0.001 -outfmt 5"
subprocess.run(cmd, shell=True)

CompletedProcess(args='blastp -query opuntia.fasta -db swissprot -out opuntia.xml -evalue 0.001 -outfmt 5', returncode=0)

In this example there shouldn’t be any output from BLASTX to the terminal. You may want to check the output file `opuntia.xml` has been created.

### 7.2.3 Other versions of BLAST
You may also come across [Washington University BLAST](http://blast.wustl.edu/) (WU-BLAST), and its successor, [Advanced Biocomputing BLAST](https://blast.advbiocomp.com) (AB-BLAST, released in 2009, not free/open source). These packages include the command line tools wu-blastall and ab-blastall, which mimicked blastall from the NCBI “legacy” BLAST suite. Biopython does not currently provide wrappers for calling these tools, but should be able to parse any NCBI compatible output from them.

## 7.3 Parsing BLAST output
we now recommend to parse the output in XML format, which can be generated by recent versions of BLAST. Not only is the XML output more stable than the plain text and HTML output, it is also much easier to parse automatically, making Biopython a whole lot more stable.

Using a GI number to do an online search:

In [13]:
from Bio.Blast import NCBIWWW
result_handle = NCBIWWW.qblast("blastn", "nt", "8332116")

If instead you ran BLAST some other way, and have the BLAST output (in XML format) in the file `my_blast.xml`, all you need to do is to open the file for reading:
* To download test file : [`my_blast.xml`](https://github.com/biopython/biopython/blob/master/Doc/examples/my_blast.xml)

In [30]:
result_handle = open("my_blast.xml")

In [25]:
from Bio.Blast import NCBIXML
blast_record = NCBIXML.read(result_handle)

or, if you have lots of results (i.e., multiple query sequences):

In [22]:
from Bio.Blast import NCBIXML
blast_records = NCBIXML.parse(result_handle)

To be able to handle the situation where the BLAST file may be huge, containing thousands of results, `NCBIXML.parse()` returns an iterator. In plain English, an iterator allows you to step through the BLAST output, retrieving BLAST records one by one for each BLAST search result:
```
>>> from Bio.Blast import NCBIXML
>>> blast_records = NCBIXML.parse(result_handle)
>>> blast_record = next(blast_records)
# ... do something with blast_record
>>> blast_record = next(blast_records)
# ... do something with blast_record
>>> blast_record = next(blast_records)
# ... do something with blast_record
>>> blast_record = next(blast_records)
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
StopIteration
# No further records
```

Or, you can use a for-loop:
```
>>> for blast_record in blast_records:
...     pass     # Do something with blast_record
...
```

Note though that you can step through the BLAST records only once. Usually, from each BLAST record you would save the information that you are interested in. If you want to save all returned BLAST records, you can convert the iterator into a list:

In [20]:
blast_records = list(blast_records)
print(blast_records)

Usually, you’ll be running one BLAST search at a time. Then, all you need to do is to pick up the first (and only) BLAST record in `blast_records`:

In [28]:
from Bio.Blast import NCBIXML
blast_records = NCBIXML.parse(result_handle)
blast_record = next(blast_records)

or more elegantly:

In [31]:
from Bio.Blast import NCBIXML
blast_record = NCBIXML.read(result_handle)

## 7.4 The BLAST record class
Let’s just print out some summary info about all hits in our blast report
greater than a particular threshold. The following code does this:

This will print out summary reports like the following:
```
****Alignment****
sequence: >gb|AF283004.1|AF283004 Arabidopsis thaliana cold acclimation protein WCOR413-like protein
alpha form mRNA, complete cds
length: 783
e value: 0.034
tacttgttgatattggatcgaacaaactggagaaccaacatgctcacgtcacttttagtcccttacatattcctc...
||||||||| | ||||||||||| || |||| || || |||||||| |||||| | | |||||||| ||| ||...
tacttgttggtgttggatcgaaccaattggaagacgaatatgctcacatcacttctcattccttacatcttcttc...
```

In [32]:
E_VALUE_THRESH = 0.04

for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print("****Alignment****")
            print("sequence:", alignment.title)
            print("length:", alignment.length)
            print("e value:", hsp.expect)
            print(hsp.query[0:75] + "...")
            print(hsp.match[0:75] + "...")
            print(hsp.sbjct[0:75] + "...")

****Alignment****
sequence: gi|1219041180|ref|XM_021875076.1| PREDICTED: Chenopodium quinoa cold-regulated 413 plasma membrane protein 2-like (LOC110697660), mRNA
length: 1173
e value: 6.87655e-117
ACAGAAAATGGGGAGAGAAATGAAGTACTTGGCCATGAAAACTGATCAATTGGCCGTGGCTAATATGATCGATTC...
|| ||||||||| |||| | |||| ||  |||| |||| | |||| ||| | |||| ||| ||| ||||| | ||...
ACCGAAAATGGGCAGAGGAGTGAATTATATGGCAATGACACCTGAGCAACTAGCCGCGGCCAATTTGATCAACTC...
****Alignment****
sequence: gi|1226796956|ref|XM_021992092.1| PREDICTED: Spinacia oleracea cold-regulated 413 plasma membrane protein 2-like (LOC110787470), mRNA
length: 672
e value: 4.33958e-113
AAAATGGGGAGAGAAATGAAGTACTTGGCCATGAAAACTGATCAATTGGCCGTGGCTAATATGATCGATTCCGAT...
|||||||| |||  |||| | || ||||| |||||||| || ||||| |||| ||| ||| ||||||||||||||...
AAAATGGGTAGACGAATGGATTATTTGGCGATGAAAACCGAGCAATTAGCCGCGGCCAATTTGATCGATTCCGAT...
****Alignment****
sequence: gi|2252585423|ref|XM_010682658.2| PREDICTED: Beta vulgaris subsp. vulgaris cold-regulated 413 plasma mem

Basically, you can do anything you want to with the info in the BLAST report once you have parsed it.
This will, of course, depend on what you want to use it for, but hopefully this helps you get started on doing what you need to do!

## 7.5 Dealing with PSI-BLAST
You can run the standalone version of PSI-BLAST (the legacy NCBI command line tool `blastpgp`, or its replacement `psiblast`) directly from the command line or using python’s subprocess module.
At the time of writing, the NCBI do not appear to support tools running a PSI-BLAST search via the internet.

## 7.6 Dealing with RPS-BLAST
You can run the standalone version of RPS-BLAST (either the legacy NCBI command line tool `rpsblast`, or its replacement with the same name) directly from the command line or using python’s subprocess module.
At the time of writing, the NCBI do not appear to support tools running an RPS-BLAST search via the internet.
You can use the `Bio.Blast.NCBIXML` parser to read the XML output from current versions of RPS-BLAST.